# [Meeting Rooms II](https://leetcode.com/problems/meeting-rooms-ii/)
## Description
Find the minimum number of conference rooms required to hold all meetings given as time intervals.
## Strategy
### Min-Heap
Use a [Min-Heap](../resources/heap.ipynb) to track when each room becomes free

### Two-Pointer
Sort start and end times separately and use a two-pointer scan to track overlap.

### Example: `[[0,30],[5,10],[15,20]]`

#### Step 1: Extract and Sort Times

```typescript
// Original: [[0,30],[5,10],[15,20]]

starts = [0, 5, 15]   // When meetings begin
ends   = [30, 10, 20] // When meetings end

// After sorting:
starts = [0, 5, 15]   // Already sorted
ends   = [10, 20, 30] // Now sorted
```

#### Step 2: Initialize Pointers

```typescript
starts = [0,  5, 15]
         ↑           startPtr = 0

ends   = [10, 20, 30]
         ↑            endPtr = 0

rooms = 0, maxRooms = 0
```

#### Step 3: Process Events Chronologically

##### Event 1: Time 0 (start)

```typescript
starts[0] = 0  vs  ends[0] = 10
0 < 10 → Meeting starts!

rooms++             // rooms = 1
maxRooms = max(1,0) // maxRooms = 1
startPtr++          // startPtr = 1

starts = [0,  5, 15]
              ↑      startPtr = 1

ends   = [10, 20, 30]
         ↑            endPtr = 0
```

**Current state**: 1 room in use

##### Event 2: Time 5 (start)

```typescript
starts[1] = 5  vs  ends[0] = 10
5 < 10 → Another meeting starts!

rooms++             // rooms = 2
maxRooms = max(2,1) // maxRooms = 2
startPtr++          // startPtr = 2

starts = [0,  5, 15]
                 ↑   startPtr = 2

ends   = [10, 20, 30]
         ↑            endPtr = 0
```

**Current state**: 2 rooms in use (peak usage!)

##### Event 3: Time 10 (end)

```typescript
starts[2] = 15  vs  ends[0] = 10
15 > 10 → Meeting ends first!

rooms--             // rooms = 1
endPtr++            // endPtr = 1

starts = [0,  5, 15]
                 ↑   startPtr = 2

ends   = [10, 20, 30]
              ↑      endPtr = 1
```

**Current state**: 1 room in use

##### Event 4: Time 15 (start)

```typescript
starts[2] = 15  vs  ends[1] = 20
15 < 20 → Meeting starts!

rooms++             // rooms = 2
maxRooms = max(2,2) // maxRooms = 2
startPtr++          // startPtr = 3 (done with starts)

starts = [0,  5, 15]
                    ↑ startPtr = 3 (beyond array)

ends   = [10, 20, 30]
              ↑      endPtr = 1
```

**Current state**: 2 rooms in use

#### Final Result: maxRooms = 2 ✅

#### Timeline Visualization

```
Time:  0    5    10   15   20   30
       |    |     |    |    |    |
       A────────────────────────→  Meeting A [0,30]
            B─────→                Meeting B [5,10]  
                        C────→     Meeting C [15,20]

Rooms: 1    2     1    2    1    0
              ↑         ↑
         Peak usage = 2 rooms needed
```

#### Summary Table

| Event | Time | Type | Action | Rooms | Max Rooms |
|-------|------|------|--------|-------|-----------|
| 1 | 0 | Start | +1 | 1 | 1 |
| 2 | 5 | Start | +1 | 2 | 2 |
| 3 | 10 | End | -1 | 1 | 2 |
| 4 | 15 | Start | +1 | 2 | 2 |

**Answer: 2 meeting rooms required**

### Chronologically tracking
Mark all start/end events and process chronologically

__Time:__ O(n log n)  
__Space:__ O(n)
 


In [5]:
import { MinHeap } from "../bin/ds/heap.ts";

// Heap = "When does each room become free?"
// peek() = "Which room becomes free earliest?"
// size() = "How many rooms are we using?"

// For each meeting:
// 1. Can we reuse the earliest-free room? → check peek()
// 2. If yes: that room is now free → extractMin()
// 3. This meeting will occupy a room until 'end' → insert(end)
// 4. Current room count = heap.size()
function minMeetingRooms(intervals: number[][]): number {
    if (intervals.length === 0) return 0;
    
    // Sort by start time
    intervals.sort((a, b) => a[0] - b[0]);
    
    // Min heap to track end times of ongoing meetings
    const endTimes: MinHeap = new MinHeap()
    
    for (const [start, end] of intervals) {
        // If earliest ending meeting has finished, reuse that room
        if (endTimes.size() > 0 && endTimes.peek() <= start) {
            endTimes.extractMin(); // Remove earliest end time (heap pop)
        }
        
        // Add current meeting's end time (heap push)
        endTimes.insert(end)
    }
    
    return endTimes.size(); // Number of rooms needed
}

// By sorting separately, we process events in chronological order:
// - If starts[i] < ends[j]: next event is a meeting starting
// - If starts[i] >= ends[j]: next event is a meeting ending
// This tells us the NET CHANGE in room usage over time!
function minMeetingRooms_twoPointer(intervals: number[][]): number {
  // split start and end times
  const starts: number[] = []
  const ends: number[] = []

  for(const [start, end] of intervals) {
    starts.push(start)
    ends.push(end)
  }

  starts.sort((a, b) => a[0] - b[0])
  ends.sort((a, b) => a[0] - b[0])

  let maxRooms: number = 0
  let rooms: number = 0
  let i: number = 0
  let j: number = 0

  while (i < starts.length) {
    if (starts[i] < ends[j]) {
      rooms++
      i++;
    } else {
      rooms--
      j++
    }
    
    maxRooms = Math.max(maxRooms, rooms)
  }

  return maxRooms
}

In [6]:
import { assertEquals } from "https://deno.land/std/testing/asserts.ts";

Deno.test("Meeting Rooms - Overlap", () => {
  assertEquals(minMeetingRooms([[0,30],[5,10],[15,20]]), 2);
});

Deno.test("Meeting Rooms - No Overlap", () => {
  assertEquals(minMeetingRooms([[7,10],[2,4]]), 1);
});



Meeting Rooms - Overlap ... ok (0ms)
Meeting Rooms - No Overlap ... ok (0ms)

ok | 2 passed | 0 failed (0ms)
